In [5]:
import numpy as np
import open3d as o3d
from loguru import logger as logger
from tqdm import tqdm

def read_ply_file(file_path):
    # points = []
    # with open(file_path, 'r') as file:
    #     lines = file.readlines()
    #     data_start_line = lines.index('end_header\n') + 1
    #     for line in lines[data_start_line:]:
    #         point = list(map(float, line.strip().split()))
    #         points.append(point)
    
    # Load the PLY file
    pcd = o3d.io.read_point_cloud(file_path)

    # Get the points
    points = np.asarray(pcd.points)
    print(f'Total of {points.size} points in the {file_path}')
    return np.array(points)

def write_ply_file(points, trajectory_points, file_path):
    with open(file_path, 'w') as file:
        file.write('ply\n')
        file.write('format ascii 1.0\n')
        file.write('element vertex {}\n'.format(len(points)))
        file.write('property float x\n')
        file.write('property float y\n')
        file.write('property float z\n')
        file.write('property uchar red\n')
        file.write('property uchar green\n')
        file.write('property uchar blue\n')
        file.write('end_header\n')
        
        # Write mesh points with default color
        for point in points[:-len(trajectory_points)]:
            file.write('{} {} {} 0 255 0\n'.format(point[0], point[1], point[2]))
        
        # Write trajectory points with red color
        for point in points[-len(trajectory_points):]:
            file.write('{} {} {} 255 0 0\n'.format(point[0], point[1], point[2]))


def concatenate_trajectory(mesh_file, trajectory_file, output_file):
    # Read mesh file
    mesh_points = read_ply_file(mesh_file)
    trajectory_points = read_trajectory(trajectory_file)
    # Concatenate trajectory points
    concatenated_points = np.concatenate((mesh_points, trajectory_points), axis=0)

    # Write new PLY file
    write_ply_file(concatenated_points, trajectory_points, output_file)


def read_trajectory(trajectory_file):
    trajectory_points = []
    # 读取视觉里程计轨迹
    trajectories = np.loadtxt(trajectory_file)
    for idx, trajectory in tqdm(enumerate(trajectories), total=len(trajectories), desc='Processing keyframes'): # 获取相机姿态
        # timestamp = trajectory[0]
        camera_pose = trajectory[1:4]
        trajectory_points.append(camera_pose)

    return trajectory_points

# # Example usage
# mesh_file = 'path/to/mesh.ply'
# trajectory_points = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
# output_file = 'path/to/output.ply'

# concatenate_trajectory(mesh_file, trajectory_points, output_file)


In [6]:
# Example usage
mesh_file = "/home/charles/Documents/zhongnan/fastlio-color/test-offline-color/test03/scans-clean.pcd"
trajectory_file = "/home/charles/Documents/zhongnan/fastlio-color/test-offline-color/test03/visual_odom.txt"
# trajectory_points = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])
output_file = '/home/charles/Documents/zhongnan/fastlio-color/test-offline-color/test03/mesh_with_pose.ply'

concatenate_trajectory(mesh_file, trajectory_file, output_file)


Total of 166973781 points in the /home/charles/Documents/zhongnan/fastlio-color/test-offline-color/test03/scans.pcd


Processing keyframes: 100%|██████████| 456/456 [00:00<00:00, 1674783.38it/s]
